<a href="https://colab.research.google.com/github/AliMotassem164/gesture-classification-cnn/blob/main/PROJECT_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)

PROJECT_ROOT = Path(".")

DATA_DIR = PROJECT_ROOT / "data" / "leapgestrecog"

CLEAN_DIR = DATA_DIR / "clean"

IMG_WIDTH, IMG_HEIGHT = 224, 224
IMG_SIZE = (IMG_WIDTH, IMG_HEIGHT)
BATCH_SIZE = 32
VAL_SPLIT = 0.2
SEED = 42

print("Project root:", PROJECT_ROOT.resolve())
print("Data dir:", DATA_DIR)
print("Clean dir (images):", CLEAN_DIR)


TensorFlow version: 2.19.0
Project root: /content
Data dir: data/leapgestrecog
Clean dir (images): data/leapgestrecog/clean


In [ ]:
!pip install -q kagglehub

import kagglehub
import shutil

RAW_DIR = DATA_DIR / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

print("Downloading dataset from Kaggle")
path = kagglehub.dataset_download("gti-upm/leapgestrecog")
print("Downloaded to:", path)

src = Path(path)
if (src / "leapGestRecog").is_dir():
    src = src / "leapGestRecog"

print("Using source folder:", src)

if not any(RAW_DIR.iterdir()):
    shutil.copytree(src, RAW_DIR, dirs_exist_ok=True)
    print(" Raw data copied to:", RAW_DIR)
else:
    print(" RAW_DIR already has data:", RAW_DIR)

if not CLEAN_DIR.exists():
    shutil.copytree(RAW_DIR, CLEAN_DIR)
    print(" Clean data folder created at:", CLEAN_DIR)
else:
    print(" Clean dir already exists:", CLEAN_DIR)


Using Colab cache for faster access to the 'leapgestrecog' dataset.
Downloaded to: /kaggle/input/leapgestrecog
Using source folder: /kaggle/input/leapgestrecog/leapGestRecog


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    CLEAN_DIR,
    labels="inferred",
    label_mode="int",
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    CLEAN_DIR,
    labels="inferred",
    label_mode="int",
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)

print(" Classes found:", class_names)
print(" Number of gesture classes:", num_classes)


In [ ]:
import shutil
import os

GESTURE_DIR = DATA_DIR / "clean_by_gesture"
GESTURE_DIR.mkdir(parents=True, exist_ok=True)

print("Target gesture folder:", GESTURE_DIR)

for subject in sorted(os.listdir(CLEAN_DIR)):
    subject_path = CLEAN_DIR / subject
    if not subject_path.is_dir():
        continue

    for gesture in os.listdir(subject_path):
        gesture_path = subject_path / gesture
        if not gesture_path.is_dir():
            continue

        target_dir = GESTURE_DIR / gesture
        target_dir.mkdir(parents=True, exist_ok=True)

        for filename in os.listdir(gesture_path):
            src_file = gesture_path / filename
            if not src_file.is_file():
                continue

            new_name = f"{subject}_{filename}"
            dst_file = target_dir / new_name

            shutil.copy2(src_file, dst_file)

print(" Done regrouping images by gesture")


In [ ]:
DATA_ROOT = GESTURE_DIR

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_ROOT,
    labels="inferred",
    label_mode="int",
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_ROOT,
    labels="inferred",
    label_mode="int",
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)

print(" Classes found:", class_names)
print(" Number of gesture classes:", num_classes)


In [ ]:
from tensorflow.keras import layers, models

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ],
    name="data_augmentation",
)

preprocess_input = tf.keras.applications.resnet50.preprocess_input

print("Data augmentation and preprocessing layers are ready ")


In [ ]:
base_model = tf.keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=IMG_SIZE + (3,)
)

base_model.trainable = False

inputs = keras.Input(shape=IMG_SIZE + (3,))

x = data_augmentation(inputs)
x = preprocess_input(x)

x = base_model(x, training=False)

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs, name="ResNet50_gesture")

model.summary()


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "resnet50_gesture_best.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1,
)

earlystop_cb = keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True,
    verbose=1,
)

EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb],
)


In [ ]:
val_loss, val_acc = model.evaluate(val_ds)
print("Final validation accuracy:", val_acc)
print("Final validation loss:", val_loss)
